In [1]:
import hyperspy.api as hs
from matplotlib import pyplot as plt
import numpy as np
import pycrystem as pc
import pymatgen as pmg
from pycrystem.indexation_generator import IndexationGenerator

plt.rcParams['image.cmap'] = 'gray'
%matplotlib tk

In [2]:
dp = pc.load("/home/pc494/Documents/data/GaN_intro/GaN_indent.blo")

In [3]:
dp.inav[348:400,76:78].plot()

In [4]:
dp = dp.inav[390:400,70:78] #selecting a thin strip at the top

In [5]:
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]));

A Jupyter Widget

In [3]:
## Set up our structure ##
Ga = pmg.Element("Ga")
N = pmg.Element("N")
lattice = pmg.Lattice.hexagonal(3.186,5.186)
structure = pmg.Structure.from_spacegroup("P63mc",lattice, [Ga,N], [[0, 0, 0],[0.5,0.5,0.5]])

NameError: name 'pmg' is not defined

In [7]:
folder = '/home/pc494/Documents/data/pattern_matching_in_perovskites/'
rot_array = np.loadtxt(folder + 'mmm_grid_euler.bin')
rot_list = rot_array.tolist()[::10] #taking every 10th rotation option
edc = pc.ElectronDiffractionCalculator(300, 0.025)
diff_gen = pc.DiffractionLibraryGenerator(edc)
struc_lib = dict()
struc_lib["GaN"] = (structure, rot_list)

In [8]:
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=1.2/128,
                                            reciprocal_radius=1.,
                                            representation='euler')

In [9]:
indexer = IndexationGenerator(dp, library)
match_results = indexer.correlate(n_largest=1)

A Jupyter Widget

/home/pc494/anaconda3/lib/python3.6/site-packages/pycrystem-0.4-py3.6-linux-x86_64.egg/pycrystem/utils/__init__.py:118: RuntimeWarning: invalid value encountered in double_scalars
  np.sqrt(np.dot(intensities_2, intensities_2))


In [10]:
match_results.plot()

In [10]:
def _convert_to_2D(peak_locations):
    #if np.any(peak_locations[:,2] > 5e-2):
    #    print("This will be a warning in future, but be aware that you may not be within a 2D regime")
    peak_locations = peak_locations[:,0:2]
    return peak_locations

In [11]:
def function_version_1(single):
    best_fit = single[0][1:4]
    best_fit_pattern = library[_phase][(best_fit[0],best_fit[1],best_fit[2])]
    peak_locations = best_fit_pattern.coordinates
    peak_locations = _convert_to_2D(best_fit_pattern.coordinates)
    return peak_locations

In [12]:
_phase = "GaN"
found_peaks = match_results.map(function_version_1,inplace=False,show_progressbar=False)

In [13]:
from pycrystem.diffraction_vectors import DiffractionVectors
found_peaks = DiffractionVectors(found_peaks)
found_peaks.axes_manager.set_signal_dimension(0)

In [ ]:
from pycrystem.utils import plot as pcplt

mmx,mmy = pcplt.generate_marker_inputs_from_peaks(found_peaks)

In [15]:
mmx.shape

(4, 10, 8)

In [17]:
dp.plot(cmap='viridis') 
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='red',marker='x')
    dp.add_marker(m,plot_marker=True,permanent=True)

Exception occurred in traits notification handler for object: x axis, trait: index, old value: 7, new value: 8
Traceback (most recent call last):
  File "/home/pc494/anaconda3/lib/python3.6/site-packages/traits/trait_notifiers.py", line 519, in _dispatch_change_event
    self.dispatch( handler, *args )
  File "/home/pc494/anaconda3/lib/python3.6/site-packages/traits/trait_notifiers.py", line 482, in dispatch
    handler( *args )
  File "/home/pc494/anaconda3/lib/python3.6/site-packages/hyperspy/axes.py", line 810, in _on_index_changed
    self.events.indices_changed.trigger(obj=self)
  File "<string>", line 4, in trigger
  File "/home/pc494/anaconda3/lib/python3.6/site-packages/hyperspy/events.py", line 402, in trigger
    function(**{kw: kwargs.get(kw, None) for kw in kwsl})
  File "/home/pc494/anaconda3/lib/python3.6/site-packages/hyperspy/drawing/image.py", line 311, in update
    marker.update()
  File "/home/pc494/anaconda3/lib/python3.6/site-packages/hyperspy/drawing/_markers/poi

In [39]:
mmx = np.transpose(mmx,[0,2,1])
print(mmx.shape)
for mx in mmx:
    print(mx.shape)
    break

(4, 152, 1)
(152, 1)


In [31]:
mmx.shape

(4, 1, 152)

(1, 152, 4)